# Duckietown and ROS

In this notebook we will use what we have learned about ROS in the previous notebooks to start to control the duckiebot, either in simulation or on real hardware. 

## See what the Duckiebot Sees

As before, launch the noVNC browser by running following command in the terminal **of your laptop** (not the terminal in VSCode!) except this time with the `--sim` argument:

    dts code workbench --sim

Note 1: You will need to stop the previous with CTRL-C and then execute this with the `--sim` option.
Note 2: This may take some time to run the first time because it is downloaded the docker image of the Duckietown simulation environment. 

Once you open the noVNC browser, as before, you should see the same icons on the left hand side. Double-click on the one that says `RQT Image ...`, clicking on this opens the [rqt_image_view](https://wiki.ros.org/rqt_image_view) ROS utility.

In the top left scroll down bar of the `rqt_image_view` window you should see only one option: `/agent/camera_node/image/compressed` if you select it you will see the output from the camera on the Duckiebot.

![rqt_image_view](../assets/rqt_image_view.png)

## Try out the Joystick

You can also open the joystick by double clicking on the `Joystick` icon. You can try clicking on the directions or using the arrow keys on your keyboard. You should see the corresponding direction on the joystick turn green, but you won't see robot moving in the simulation (assuming you still have the `rqt_image_view` window open from the previous step).

However, the joystick **is** doing something. To see what it is doing, open up a terminal by clicking on the `LXTerminal` icon as before. Execute the following command on the terminal:

    rostopic echo /agent/joy

The click back to the joystick window and start clicking on the directions and look closely at the output that is produced in the terminal. In particular, look at the `axes` part of the data that is produced. You should see one of the values changing (the position may change and the value may be either a `+1` or a `-1` depending on which direction you push).

![joy_echo](../assets/joy_echo.png)

Finally, do the following in the terminal

    rostopic list

to see all of the topics:

```
/agent/camera_node/camera_info
/agent/camera_node/image/compressed
/agent/coordinator_node/intersection_go
/agent/episode_start
/agent/joy
/agent/led_emitter_node/led_pattern
/agent/left_wheel_encoder_node/tick
/agent/right_wheel_encoder_node/tick
/agent/wheels_driver_node/emergency_stop
/agent/wheels_driver_node/wheels_cmd
/rosout
/rosout_agg
```

we can see the topic `/agent/joy` and we know at least how that data is being published. This message is of type [Joy.msg](https://docs.ros.org/en/noetic/api/sensor_msgs/html/msg/Joy.html) (this is a standard datatype provided by ROS). The topic that we need to publish messages on to make the robot actually move is `/agent/wheels_driver_node/wheels_cmd`. This message is of type [WheelsCmdStamped.msg](https://github.com/duckietown/dt-ros-commons/blob/daffy/packages/duckietown_msgs/msg/WheelsCmdStamped.msg) (this is a message that we defined and is available to us because it is defined in an upstream Docker image).

### The Task Ahead

Our objective for the rest of this exercise will be to create a ROS node to take the messages that are being published on the `/agent/joy` topic and use them to publish something to the `/agent/wheels_driver_node/wheels_cmd` topic so that the robot moves and we can control it with the joystick.  

## Create a New Package

Use the menu on the left of this notebook to open a new terminal (three lines -> Terminal -> New Terminal). In that terminal, navigate to the `packages` directory:

    cd packages

Once there we can proceed to create our package. Let's call it `dt-joystick-demo`. You can create it by executing the following code in your terminal

    catkin_create_pkg dt-joystick-demo std_msgs sensor_msgs duckietown_msgs rospy

You should see the output:

```
Created file dt-joystick-demo/package.xml
Created file dt-joystick-demo/CMakeLists.txt
Successfully created files in /code/lx-ros-basics/packages/dt-joystick-demo. Please adjust the values in package.xml.
```

let's enter into the `dt-joystick-demo` directory:

    cd dt-joystick-demo

then make a `src` dirctory:

    mkdir src

now enter that `src` directory:

    cd src

First we need to create the file `__init__.py`

    touch __init__.py

and then create a python file:

    touch dt-joystick-demo-node.py

and finally we need to make that file execuatable:

    chmod +x dt-joystick-demo-node.py

You should now be able to navigate to that file using the file directory on the left. Click on the file to open it (you may want to split screen using the button at the top right ). 

## Writing our ROS Node

We need to start with some basics. First off we need the following at the top of the file:

```python
#!/usr/bin/env python3
```

This tells the python interpreter that this is a python file. 

Next we will need to import some packages, these will by `rospy` and then the types of *messages* we will be using later.

```python
import rospy
from sensor_msgs.msg import Joy
from duckietown_msgs.msg import WheelsCmdStamped
```

At the bottom of your file let's build the main function which will be called when your python script is executed. It should look something like this:

```python
if __name__ == "__main__":
    # Initialize the node
    node = DTJoystickDemoNode()
    rospy.init_node('dt-joystick-demo-node')
    # Keep it spinning
    rospy.spin()
```

This code initializes a class of type `DTJoystickDemoNode` (which we have yet to define), and then does some ROS-specific stuff, namely initializes the node with the `rospy.init_node` which registers the node with a unique name, and then finally starts the node running with the `rospy.spin()` function. This will ensure that we get data from topics we subscribe to and that data that we publish actually gets sent (among other things).

Now we are ready to define the class called `DTJoystickDemoNode`. We can start with this code:

```python
class DTJoystickDemoNode():
    def __init__(self):
```

### Building the publisher and subscriber

The main thing we need to do in the `__init__` function is setup the publisher and subscriber. For the subscriber this should look something like this:

```python
        self.sub_joy = rospy.Subscriber(
            "/agent/joy", 
            Joy,
            self.process_joy
        )
```

Note: Make sure to get the indentation right

This initializes a `Subscriber`, tells it to listen to the topic `agent/joy`, tells it that the data coming on the topic should be of type `Joy`, and that we will process it in a function `self.process_joy` (this is called a function callback). 

The publisher should look like this:

```python
        self.pub_wheel_cmds = rospy.Publisher(
            "/agent/wheels_driver_node/wheels_cmd",
            WheelsCmdStamped
        )
```

This initializes a `Publisher` that will publish data of type `WheelsCmdStamped` onto a topic called `agent/wheels_driver_node/wheels_cmd`



### Processing the incoming data

The last task we need to do is to build this function called `process_joy` that will process the incoming messages. Start by defining the function:

```python
    def process_joy(self,msg):
```

Notice that in addition to the normal `self` variable for a function that is defined in a class, there is also the `msg` variable. This is the variable that will contain the data. 

To make the code as clean as possible, let's start by defining and initializing the variable that we are going to end up publishing

```python
        cmd_to_publish = WheelsCmdStamped()
        cmd_to_publish.header = msg.header
        cmd_to_publish.vel_right = 0.0
        cmd_to_publish.vel_left = 0.0
```

The `header` part of the variable contains some meta information, such as a timestamp. It is good practice to copy this over from the incoming data so that we can calculate things like latency, but don't worry too much about this for now. As we say from the definition of the [WheelsCmdStamped.msg](https://github.com/duckietown/dt-ros-commons/blob/daffy/packages/duckietown_msgs/msg/WheelsCmdStamped.msg) message, we need to set a `vel_right` and a `vel_left` (for each of the two wheels on the robot). We can start by initializing those to zero so that in the case that not of the joystick buttons are active we will not move. 

It might also be helpful to define a variable that will be the velocity that we are going to set the wheels to. 

```python
        v = 0.5 # this will be the velocity we publish
```

Now we just need to check which of the buttons was pressed and then set the correct values (either + or - `v`) to the corresponding wheels to make the robot go forward or backwards or turn left or right. Recall from the previous investigation that this information comes in on the `axes` data with the [Joy.msg](https://docs.ros.org/en/noetic/api/sensor_msgs/html/msg/Joy.html). Specifically when we did the earlier test, we noticed that pushing forwards makes the second number in that array become `1.0`, pushing backwards makes the second number in that array become `-1.0`, pushing the right arrow makes the fourth number in the array become `-1.0`, and pushing the left hand arrow makes the fourthe number in the array become `1.0`. Using that information and our understanding of how the robot should move we can define the velocities that we should set on the wheels:

```python
        if msg.axes[1] == 1.0:
            # go forward
            cmd_to_publish.vel_right = v
            cmd_to_publish.vel_left = v

        if msg.axes[1] == -1.0:
            # go backwards
            cmd_to_publish.vel_right = -v
            cmd_to_publish.vel_left = -v

        if msg.axes[3] == -1.0:
            # turn right
            cmd_to_publish.vel_right = -v
            cmd_to_publish.vel_left = v

        if msg.axes[3] == 1.0:
            # turn left
            cmd_to_publish.vel_right = v
            cmd_to_publish.vel_left = -v
```



### Publishing the data

And finally the last thing we need to do is to publish the data. We can use the `Publisher` that we defined earlier:

```python
        self.pub_wheel_cmds.publish(cmd_to_publish)
```




### Testing if the Code Builds

In your terminal **in this notebook** you can test if your code doesn't have any syntax errors by navigating to the `packages` directory and running

    catkin build

If all goes well, you should an output like this:

```bash
------------------------------------------------------------------
Profile:                     default
Extending:          [cached] /code/catkin_ws/devel:/opt/ros/noetic
Workspace:                   /code/lx-ros-basics/packages
------------------------------------------------------------------
Build Space:        [exists] /code/lx-ros-basics/packages/build
Devel Space:        [exists] /code/lx-ros-basics/packages/devel
Install Space:      [unused] /code/lx-ros-basics/packages/install
Log Space:          [exists] /code/lx-ros-basics/packages/logs
Source Space:       [exists] /code/lx-ros-basics/packages/src
DESTDIR:            [unused] None
------------------------------------------------------------------
Devel Space Layout:          linked
Install Space Layout:        None
------------------------------------------------------------------
Additional CMake Args:       None
Additional Make Args:        None
Additional catkin Make Args: None
Internal Make Job Server:    True
Cache Job Environments:      False
------------------------------------------------------------------
Buildlisted Packages:        None
Skiplisted Packages:         None
------------------------------------------------------------------
Workspace configuration appears valid.
------------------------------------------------------------------
[build] Found 1 packages in 0.0 seconds.                                                                                                                                   
[build] Package table is up to date.                                                                                                                                       
Starting  >>> dt-joystick-demo                                                                                                                                             
Finished  <<< dt-joystick-demo                [ 3.1 seconds ]                                                                                                              
[build] Summary: All 1 packages succeeded!                                                                                                                                 
[build]   Ignored:   None.                                                                                                                                                 
[build]   Warnings:  None.                                                                                                                                                 
[build]   Abandoned: None.                                                                                                                                                 
[build]   Failed:    None.                                                                                                                                                 
[build] Runtime: 3.2 seconds total.
```


## Testing your code in the simulator

In the terminal on your laptop, stop the previous `dts code workbench` from running and restart while adding the `--launcher joystick` parameter at the end:

    dts code workbench --sim --launcher joystick

Although it is a little opaque for now, this `--launcher joystick` will run your `dt-joystick-demo-node.py` node (as long as you called the package and the node exactly what was specified above). 

Open up the noVNC browser as before. Open up `rqt_image_view` and the joystick and try holding down buttons on the joystick. You should see that you can control your Duckiebot in the simulation with the joystick. 





## Testing your code on a real Duckiebot

You should boot up your Duckiebot and make sure that you can connect to it. Run 

    dts code workbench --duckiebot [YOUR_DUCKIEBOT_NAME ]